<a href="https://colab.research.google.com/github/Ziyan-jia/PETS/blob/main/Copy_of_Pets_nn_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib notebook
!python -m pip install mpi4py
!python -m pip install mbrl
import torch
import numpy as np
import matplotlib.pyplot as plt
import time
import torch.nn as nn
import torch.optim as optim
import math
import random
from torch.autograd import Variable
import torch.nn.functional as F
import pandas as pd
import mbrl.util.math
from sklearn import preprocessing
from mpi4py import MPI 
from sklearn.preprocessing import StandardScaler
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
boss = rank==0
# Seed RNG
np.random.seed(rank)
# python -m pip install mpi4py

     |████████████████████████████████| 2.5 MB 6.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.3-cp37-cp37m-linux_x86_64.whl size=2185293 sha256=da4b6d32e205bd3ba9ec386824ab5a28150909a204e7c49d4400d5d8c545ee9d
  Stored in directory: /root/.cache/pip/wheels/7a/07/14/6a0c63fa2c6e473c6edc40985b7d89f05c61ff25ee7f0ad9ac
Successfully built mpi4py
     |████████████████████████████████| 134 kB 8.1 MB/s 
     |████████████████████████████████| 2.3 MB 62.3 MB/s 
     |████████████████████████████████| 11.2 MB 51.6 MB/s 
     |████████████████████████████████| 297 kB 59.8 MB/s 
     |████████████████████████████████| 122 kB 71.2 MB/s 
     |████████████████████████████████| 1.6 MB 53.2 MB/s 
     |████████████████████████████████| 3.3 MB 52.5 MB/s 
     |████████████████████████████████| 74 kB 4.4 MB/s 
     |████████████████████████████████| 112 kB 79.2 MB/s 
 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [65]:
training_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ziyan/ILQR/training data.csv')
test_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ziyan/ILQR/test data.csv')
valid_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ziyan/ILQR/valid data.csv')
training_data = np.array(training_data)
testt_data = np.array(test_data)
test_data = np.array(test_data)
valid_data = np.array(valid_data)

In [66]:
print(training_data[0,:])
scalerX = StandardScaler().fit(training_data[:,7:14])
scalery = StandardScaler().fit(training_data[:,-7:])
training_data[:,7:14] = scalerX.transform(training_data[:,7:14])
training_data[:,-7:] = scalery.transform(training_data[:,-7:])
test_data[:,7:14] = scalerX.transform(test_data[:,7:14])
test_data[:,-7:] = scalery.transform(test_data[:,-7:])
valid_data[:,7:14] = scalerX.transform(valid_data[:,7:14])
valid_data[:,-7:] = scalery.transform(valid_data[:,-7:])
print(training_data[0,:])

[  1.3467995    1.0224129   -1.7369454    0.98558265   0.7130162
  -0.70451635   0.04044396 -10.758503   -10.593262    -2.9916844
 -21.944834     6.845678     5.5541124  -14.982534    12.548387
 -39.394768    37.680687   -34.75261    -33.981106    33.326324
  -0.75313205   1.2910267    0.9443954   -1.8109274    0.6838323
   0.6794432   -0.41811532   0.09664377  -5.577281    -7.801746
  -7.398209   -30.175035    -3.357305    28.6401       5.6199813 ]
[  1.3467995    1.0224129   -1.7369454    0.98558265   0.7130162
  -0.70451635   0.04044396  -0.63276274  -0.61573357  -0.18090254
  -1.30396275   0.40756423   0.33634883  -0.83762617  12.548387
 -39.394768    37.680687   -34.75261    -33.981106    33.326324
  -0.75313205   1.2910267    0.9443954   -1.8109274    0.6838323
   0.6794432   -0.41811532   0.09664377  -0.28264525  -0.45192007
  -0.37051383  -1.41322991  -0.11031186   0.91180135   0.27711511]


In [4]:
print(training_data[0,:])
scalerX = StandardScaler().fit(training_data[:,:21])
scalery = StandardScaler().fit(training_data[:,-14:])
training_data[:,:21] = scalerX.transform(training_data[:,:21])
training_data[:,-14:] = scalery.transform(training_data[:,-14:])
test_data[:,:21] = scalerX.transform(test_data[:,:21])
test_data[:,-14:] = scalery.transform(test_data[:,-14:])
valid_data[:,:21] = scalerX.transform(valid_data[:,:21])
valid_data[:,-14:] = scalery.transform(valid_data[:,-14:])
print(training_data[0,:])

[  1.3467995    1.0224129   -1.7369454    0.98558265   0.7130162
  -0.70451635   0.04044396 -10.758503   -10.593262    -2.9916844
 -21.944834     6.845678     5.5541124  -14.982534    12.548387
 -39.394768    37.680687   -34.75261    -33.981106    33.326324
  -0.75313205   1.2910267    0.9443954   -1.8109274    0.6838323
   0.6794432   -0.41811532   0.09664377  -5.577281    -7.801746
  -7.398209   -30.175035    -3.357305    28.6401       5.6199813 ]
[ 1.13583613  0.88480469 -1.52811344  0.845197    0.61552332 -0.60419242
  0.04821724 -0.63276274 -0.61573357 -0.18090254 -1.30396275  0.40756423
  0.33634883 -0.83762617  0.67264551 -1.33067774  1.63294522 -1.18465577
 -1.10318896  1.49558414  0.16841617  1.06998401  0.81092974 -1.56790541
  0.61283708  0.58413752 -0.39615061  0.14918687 -0.28264525 -0.45192007
 -0.37051383 -1.41322991 -0.11031186  0.91180135  0.27711511]


In [ ]:
#index = [7,8,9,10,11,12,13,34,33,32,31,30,29,28]
index = [34,33,32,31,30,29,28]
print(training_data[0,:])
for i in range(3000):
  for j in index:
    training_data[i,j] = training_data[i,j] / 10
for i in range(3000):
  for j in index:
    test_data[i,j] = test_data[i,j] / 10
for i in range(3000):
  for j in index:
    valid_data[i,j] = valid_data[i,j] / 10
print(training_data[0,:])

[  1.3467995    1.0224129   -1.7369454    0.98558265   0.7130162
  -0.70451635   0.04044396 -10.758503   -10.593262    -2.9916844
 -21.944834     6.845678     5.5541124  -14.982534    12.548387
 -39.394768    37.680687   -34.75261    -33.981106    33.326324
  -0.75313205   1.2910267    0.9443954   -1.8109274    0.6838323
   0.6794432   -0.41811532   0.09664377  -5.577281    -7.801746
  -7.398209   -30.175035    -3.357305    28.6401       5.6199813 ]
[  1.3467995    1.0224129   -1.7369454    0.98558265   0.7130162
  -0.70451635   0.04044396 -10.758503   -10.593262    -2.9916844
 -21.944834     6.845678     5.5541124  -14.982534    12.548387
 -39.394768    37.680687   -34.75261    -33.981106    33.326324
  -0.75313205   1.2910267    0.9443954   -1.8109274    0.6838323
   0.6794432   -0.41811532   0.09664377  -0.5577281   -0.7801746
  -0.7398209   -3.0175035   -0.3357305    2.86401      0.56199813]


In [67]:
class Model(torch.nn.Module):
  def __init__(self, input_dim, output_dim, learning_rate, seed):

    super(Model, self).__init__()
    torch.manual_seed(seed)
    self.input_dim = input_dim
    self.output_dim = output_dim
    self.hidden_units = 200

    self.min_logvar = nn.Parameter(
                -10 * torch.ones(1, self.output_dim//2), requires_grad=False
            )
    self.max_logvar = nn.Parameter(
                0.5 * torch.ones(1, self.output_dim//2), requires_grad=False
            )
    #self.learn_logvar_bounds: bool = False

    
        # Instantiate model
    self.model = torch.nn.Sequential(
    torch.nn.Linear(self.input_dim, self.hidden_units, bias=True),
    torch.nn.ReLU(),
    torch.nn.Linear(self.hidden_units, self.hidden_units, bias=True),
    torch.nn.ReLU(),
    #torch.nn.Linear(self.hidden_units, self.hidden_units, bias=True),
    #torch.nn.ReLU(),
    #torch.nn.Linear(self.hidden_units, self.hidden_units, bias=True),
    #torch.nn.ReLU(),
    torch.nn.Linear(self.hidden_units, self.hidden_units, bias=True),
    torch.nn.ReLU(),
    torch.nn.Linear(self.hidden_units, self.output_dim, bias=True)
    ).to(torch.device('cpu'))
    # Instantiate optimizer
    self.optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)

  def adjust_learning_rate(self):
    for param_group in self.optimizer.param_groups:
      param_group['lr'] = param_group['lr']*0.999

  def softplus(self, x):
    """ Compute softplus """
    softplus = torch.log(1+torch.exp(x))
    # Avoid infinities due to taking the exponent
    softplus = torch.where(softplus==float('inf'), x, softplus)
    return softplus

  def NLL(self, mean, var, truth):
    #diff = torch.sub(truth, mean)
    #var = self.softplus(var)
      # Compute loss 
    #loss = torch.mean(torch.div(diff**2, 2*var))
    #loss += torch.mean(0.5*torch.log(var))
    loss = (mbrl.util.math.gaussian_nll(mean, var, truth, reduce=False)
         .mean(axis=0)  # average over batch and target dimension
         .sum())
         # sum over ensemble dimension
    loss += 0.01 * (self.max_logvar.sum() - self.min_logvar.sum())
    return loss

  def forward(self, inputs, input_type):
    """ Forward pass for a given input
        :input:     (state,action)-pair
        :returns:   means and variances given input
    """
    # Compute output of model
    if input_type == "nparray":            
      x = torch.from_numpy(inputs).float()
      x = x.view(-1,self.input_dim)
      x = x.cuda()
      out = self.model(x)
      mean, var = torch.split(out, self.output_dim//2, dim=1)
      #var = self.softplus(var)
      var = self.max_logvar - F.softplus(self.max_logvar - var)
      var = self.min_logvar + F.softplus(var - self.min_logvar)
      #mean.detach().numpy(), var.detach().numpy()
      return mean, var
    else:
      inputs = inputs.view(-1,self.input_dim)
      inputs = inputs.cuda()
      out = self.model(inputs)
      mean, var = torch.split(out, self.output_dim//2, dim=1)
      var = self.max_logvar - F.softplus(self.max_logvar - var)
      var = self.min_logvar + F.softplus(var - self.min_logvar)
      return mean, var
      
  def step(self, inputs, true_out):
    """ Execute gradient step given the samples in the minibatch """
    # Convert input and true_out to useable tensors
    #x = torch.from_numpy(inputs).float()
    #y = torch.from_numpy(true_out).float()
    x = inputs
    y = true_out  

    # Compute output of model
    out = self.model(x)
    mean, var = torch.split(out, self.output_dim//2, dim=1)
    var = self.max_logvar - F.softplus(self.max_logvar - var)
    var = self.min_logvar + F.softplus(var - self.min_logvar)
    


    # Compute loss 
    self.nll = self.NLL(mean, var, y)

    # Backpropagate the loss
    self.optimizer.zero_grad()
    self.nll.backward()
    torch.nn.utils.clip_grad_norm_(self.model.parameters(), 10)
    self.optimizer.step()

    self.adjust_learning_rate()

  def compute_errors(self, train_in, validation_in, test_in):
    """ Compute loss on the training, validation and test data """
    # Training data
    train_in = torch.from_numpy(train_in).float()
    train_in = train_in.cuda()
    train_in, train_out = torch.split(train_in, [21,14], dim=1)
    mean, var = self.forward(train_in, "tensor")
    train_loss = self.NLL(mean, var, train_out).item()

    # Validation data
    validation_in = torch.from_numpy(validation_in).float()
    validation_in = validation_in.cuda()
    validation_in, val_out = torch.split(validation_in, [21,14], dim=1)
    mean, var = self.forward(validation_in, "tensor")
    val_loss = self.NLL(mean, var, val_out).item()

    # Test data
    test_in = torch.from_numpy(test_in).float()
    test_in = test_in.cuda()
    test_in, test_out = torch.split(test_in, [21,14], dim=1)
    mean, var = self.forward(test_in, "tensor")
    test_loss = self.NLL(mean, var, test_out).item()

    return train_loss, val_loss, test_loss

In [68]:
ensemble_size = 4          # Ensemble size per core
epochs = 15000
learning_rate = 0.0001
training_samples = 3000
validation_samples = training_samples
test_samples = 3000
batch_size = 128
measurements = epochs//200  # Measure every n steps

# Define network architecture
input_dim = 21
output_dim = 28
    # Define systems
    # current available data sets: SimpleTwoDimensional(N, seed)
datamic_systems = ["linear"]

seeds = np.random.randint(1e4, size=ensemble_size)
    # Allocate for losses
train_error = np.zeros((ensemble_size, measurements))
validation_error = np.zeros(train_error.shape)
test_error = np.zeros(train_error.shape)
    
for system in datamic_systems:
        # Instantiate class objects
        # data = datamics.datamics(training_samples, system)
        # Initialize models
  ensemble = [Model(input_dim, output_dim, learning_rate, seeds[i]) for i in range(ensemble_size)]
      # Initialize & allocate
  ensemble_mean = np.zeros((test_samples, 14))
  ensemble_var = np.zeros((test_samples, 14))

      

  # Train an ensemble of probabilistic networks:
  i = 0
  for model in ensemble:
    model = model.cuda()
    j = 0
    for epoch in range(epochs):
      indices = np.random.choice(range(training_samples-1), size=batch_size,replace=False)
      minibatch=training_data[indices]
      model_in = minibatch[:,:21]
      y = minibatch[:,21:]
      model_in = torch.from_numpy(model_in)
      y = torch.from_numpy(y).float()
      model_in = model_in.cuda().float()
      y = y.cuda()
      model.step(model_in, y)

      #scheduler.step()
      if (epoch+1)%(epochs//measurements)==0:
        # Compute error on train, test and validation sets
        train_error[i,j], validation_error[i,j], test_error[i,j] = model.compute_errors(training_data, valid_data, test_data)
        j += 1
      #if (epoch+1)%100 == 0:
        print('Epoch %s, Train loss %s, Valid loss %s, Test loss %s'%(epoch, train_error[i,j-1],validation_error[i,j-1], test_error[i,j-1]))

      # Test model on training samples

  #mean, var = model.forward(test_data[:,:21], "nparray")


      # Add to ensemble mean and variance
  #ensemble_mean += mean 
  #ensemble_var += var + mean**2

  i += 1

Epoch 199, Train loss 14.207433700561523, Valid loss 15.43763256072998, Test loss 15.226951599121094
Epoch 399, Train loss 13.387285232543945, Valid loss 15.251978874206543, Test loss 14.989690780639648
Epoch 599, Train loss 12.690532684326172, Valid loss 15.254193305969238, Test loss 14.932465553283691
Epoch 799, Train loss 12.02305793762207, Valid loss 15.024203300476074, Test loss 14.777741432189941
Epoch 999, Train loss 11.3355712890625, Valid loss 14.515841484069824, Test loss 14.218672752380371
Epoch 1199, Train loss 10.687066078186035, Valid loss 14.370747566223145, Test loss 14.049384117126465
Epoch 1399, Train loss 10.066839218139648, Valid loss 13.978588104248047, Test loss 13.598421096801758
Epoch 1599, Train loss 9.50196361541748, Valid loss 13.667015075683594, Test loss 13.271349906921387
Epoch 1799, Train loss 9.002838134765625, Valid loss 13.528419494628906, Test loss 13.123940467834473
Epoch 1999, Train loss 8.589351654052734, Valid loss 13.023871421813965, Test loss 12

In [80]:
def simulate_system(x, u):
  ensemble_mean = np.zeros((1,14))
  #ensemble_mean = torch.from_numpy(ensemble_mean)
  #ensemble_mean = ensemble_mean.cuda()
  ensemble_var = np.zeros((1,14))
  #ensemble_var = torch.from_numpy(ensemble_var)
  #ensemble_var = ensemble_var.cuda()

  x = x.T
  u = u.T
  x = np.hstack((x,u))
  x = x.astype(np.float32)
  #x[7:14]=x[7:14]/8
  #x = torch.from_numpy(x)
  x=x.reshape(1, -1)
  #scaler = StandardScaler().fit(x)
  x[:,7:14] = scalerX.transform(x[:,7:14])
  for model in ensemble:
    mean, var = model.forward(x, "nparray")
    mean = mean.cpu()
    var = var.cpu()
    mean = mean.detach().numpy()
    mean[:,-7:] = scalery.inverse_transform(mean[:,-7:])
    var = var.detach().numpy()
    ensemble_mean += mean
    ensemble_var += var + mean ** 2
    
  ensemble_mean = ensemble_mean / ensemble_size
  ensemble_var = ensemble_var / ensemble_size
  ensemble_var = ensemble_var - ensemble_mean**2
  #ensemble_mean = ensemble_mean.cpu()
  #ensemble_var = ensemble_var.cpu()
  #ensemble_mean = ensemble_mean.detach().numpy()
  #ensemble_mean[0,-7:]=ensemble_mean[0,-7:]*10
  #print(ensemble_mean)

  return ensemble_mean,ensemble_var

In [78]:
def simulate_system(x, u):
  ensemble_mean = np.zeros((1,14))
  #ensemble_mean = torch.from_numpy(ensemble_mean)
  #ensemble_mean = ensemble_mean.cuda()
  ensemble_var = np.zeros((1,14))
  #ensemble_var = torch.from_numpy(ensemble_var)
  #ensemble_var = ensemble_var.cuda()

  x = x.T
  u = u.T
  x = np.hstack((x,u))
  x = x.astype(np.float32)
  #x[7:14]=x[7:14]/8
  #x = torch.from_numpy(x)
  #x=x.reshape(1, -1)
  #scaler = StandardScaler().fit(x)
  #x = scalerX.transform(x)
  for model in ensemble:
    mean, var = model.forward(x, "nparray")
    mean = mean.cpu()
    var = var.cpu()
    mean = mean.detach().numpy()
    #mean = scalery.inverse_transform(mean)
    var = var.detach().numpy()
    ensemble_mean += mean
    ensemble_var += var + mean ** 2
    
  ensemble_mean = ensemble_mean / ensemble_size
  ensemble_var = ensemble_var / ensemble_size
  ensemble_var = ensemble_var - ensemble_mean**2
  #ensemble_mean = ensemble_mean.cpu()
  #ensemble_var = ensemble_var.cpu()
  #ensemble_mean = ensemble_mean.detach().numpy()
  #ensemble_mean[0,-7:]=ensemble_mean[0,-7:]*10
  #print(ensemble_mean)

  return ensemble_mean,ensemble_var

In [72]:
num = random.sample(range(1, 1000), 100)
output = 0
for i in num:
  ZZ = np.array(test_data[i,:14])
  UU = np.array(test_data[i,14:21])
  mean, var = simulate_system(ZZ, UU)
  mseloss = nn.MSELoss()
  target = torch.from_numpy(test_data[i,21:])
  target = target.view(-1,14)
  target = target.cuda()
  output += mseloss(torch.from_numpy(mean).cuda(), target)
output = output / 100
print(output)

tensor(0.7021, device='cuda:0', dtype=torch.float64)


In [77]:
num = random.sample(range(1, 1000), 100)
output = 0
for i in num:
  ZZ = np.array(testt_data[i,:14])
  UU = np.array(testt_data[i,14:21])
  mean, var = simulate_system(ZZ, UU)
  mseloss = nn.MSELoss()
  target = torch.from_numpy(testt_data[i,21:])
  target = target.view(-1,14)
  target = target.cuda()
  output += mseloss(torch.from_numpy(mean).cuda(), target)
output = output / 100
print(output)

tensor(155.1498, device='cuda:0', dtype=torch.float64)


In [79]:
ZZ = np.array(test_data[10,:14])
UU = np.array(test_data[10,14:21])
mean, var = simulate_system(ZZ, UU)
mseloss = nn.MSELoss()
target = torch.from_numpy(test_data[10,21:])
target = target.view(-1,14)
target = target.cuda()
output = mseloss(torch.from_numpy(mean).cuda(), target)
print(mean)
#mean[0,-7:]=mean[0,-7:]*8
#print(mean)
print(test_data[10,21:])
print(output)

[[-0.41994518  0.32729205  0.18088663 -0.27017845  0.29406992  0.1758893
   0.19786687  0.29198781 -0.10809198  0.16674505  0.51833213  0.52414673
   0.1959481   0.3832396 ]]
[-0.12915713  1.6808405   1.0073076  -1.5112218   1.9002388   0.6560098
  0.20049791  0.46418228 -1.56467176 -0.49174439  0.75359988  1.14326798
  0.52984028  1.20908749]
tensor(0.7691, device='cuda:0', dtype=torch.float64)


In [81]:
   
def get_linearization(current_state, control_signal):
    eps = 1e-5
    A = np.zeros([len(current_state), len(current_state)])
    for ii in range(len(current_state)):
        x = current_state.copy()
        x[ii] += eps
        x_inc,varr = simulate_system(x, control_signal)  
        x = current_state.copy()
        x[ii] -= eps
        x_dec,varr = simulate_system(x, control_signal)
        A[:,ii] = (x_inc - x_dec) / (2 * eps)
 
    B = np.zeros([len(current_state), len(control_signal)])
    for ii in range(len(control_signal)):
        u = control_signal.copy()
        u[ii] += eps
        x_inc,varr = simulate_system(current_state, u)
        u = control_signal.copy()
        u[ii] -= eps
        x_dec,varr = simulate_system(current_state, u)
        B[:,ii] = (x_inc - x_dec) / (2 * eps)
    return A, B

def solve_ricatti_equations(Z,U,Q,R,N,q,r):
    
    K_gains = []
    k_feedforward = []
    S = Q[-1]
    s = q[-1]
       
    for i in reversed(range(horizon_length)):
        A, B = get_linearization(Z[:,i], U[:,i])
        W = np.linalg.inv(Sigma*var)-sigma*C.T.dot(S).dot(C)
        iW = np.linalg.inv(W)
        term_covar = C.dot(iW).dot(C.T)
        H = R + B.T.dot(S).dot(B) + sigma * B.T.dot(S.T).dot(term_covar).dot(S).dot(B)
        g = r[i] + B.T.dot(s) + sigma * B.T.dot(S.T).dot(term_covar).dot(s)
        G = B.T.dot(S).dot(A) + sigma * B.T.dot(S.T).dot(term_covar.T).dot(S).dot(A)
        k = -np.linalg.solve(H,g)
        K = -np.linalg.solve(H,G)
        s = q[i] + A.T.dot(s) + G.T.dot(k) + K.T.dot(g) + K.T.dot(H).dot(k) + sigma * A.T.dot(S.T).dot(term_covar).dot(s)
        S = Q[i] + A.T.dot(S).dot(A) + K.T.dot(H).dot(K) + G.T.dot(K) + K.T.dot(G) + sigma * A.T.dot(S.T).dot(term_covar).dot(S).dot(A)
      
        K_gains.append(K)
        k_feedforward.append(k)
        
        
       
    K_gains = K_gains[::-1]
    k_feedforward = k_feedforward[::-1]
    return K_gains, k_feedforward

def controller(Z,U,q,r,horizon_length,Q,R,alpha):
    state2=np.empty([14, horizon_length+1])
    z0 = np.zeros([14,])
    state2[:,0] = z0
    u2 = np.zeros([7, horizon_length])
    K,k = solve_ricatti_equations(Z,U,Q,R,horizon_length,q,r)
    for i in range(horizon_length):
        u2[:,i] = U[:,i] + K[i] @ (state2[:,i] - Z[:,i])+ alpha*k[i]
        state2[:,i+1],var = simulate_system(state2[:,i], u2[:,i])
    state=state2.copy()
    u=u2.copy()
    return state,u,k

def compute_cost(Z,U,z_bar,u_bar, horizon_length):
    J=((Z[:,horizon_length]-z_bar[:,horizon_length]).T)@ Q[horizon_length] @(Z[:,horizon_length]-z_bar[:,horizon_length])
    for i in range(horizon_length):
        J=J+((Z[:,i]-z_bar[:,i]).T)@ Q[i] @(Z[:,i]-z_bar[:,i])+((U[:,i]-u_bar[:,i]).T @ R @ (U[:,i]-u_bar[:,i]))
    
    return J

def get_quadratic_approximation_cost(Z,U, horizon_length,z_bar,u_bar):
    q=[]
    r=[]
    J=compute_cost(Z,U,z_bar,u_bar, horizon_length)
    #J=J+0.5*((z[:,horizon_length]-Z[:,horizon_length]).T)@ Q[horizon_length] @(z[:,horizon_length]-Z[:,horizon_length])+2*(z[:,horizon_length].T-z_bar[:,horizon_length].T)@Q[horizon_length]@(z[:,horizon_length]-Z[:,horizon_length])
    for i in range(horizon_length+1):
        #J=J+0.5*((u[:,i]-U[:,i]).T)@ R @(u[:,i]-U[:,i])+0.5*((z[:,i]-Z[:,i]).T)@ Q[i] @(z[:,i]-Z[:,i])+2*((z[:,i]-z_bar[:,i]).T)@Q[i]@(z[:,i]-Z[:,i])+2*((u[:,i]-u_bar[:,i]).T)@R@(u[:,i]-U[:,i])    
        q1=2*(Z[:,i]-z_bar[:,i]).T@Q[i]
        q.append(q1)
    for i in range(horizon_length):
        r1=2*(U[:,i]-u_bar[:,i]).T@R
        r.append(r1)
    
    return q,r

In [82]:
a=1
horizon_length = 40
Q2=0.00001*np.eye(14)
Q1=30*np.eye(14)
for i in range(7,14):
    Q1[i,i]=0.00001
Q=[]
for i in range(horizon_length+1):
    if (i==horizon_length):
        Q.append(Q1)
    else:
        Q.append(Q2)

R=0.00001*np.eye(7)
sigma=0
C=np.eye(14)
Sigma=1*np.eye(14)
var = np.ones((14,1))


u_bar = np.zeros([7, horizon_length])
U = np.zeros([7, horizon_length])
Z = np.zeros([14,horizon_length+1])
u= np.zeros([7, horizon_length])
    
      
z_bar=np.zeros([14,horizon_length+1])
z_bar[0:7,horizon_length]=np.array([-1.8336430453226058, -1.4900804580013904, -0.33423961967431476, -1.108836361791467, 0.7815563306180698, 0.5042580934421823, 1.1407696303315698])

In [83]:
q,r=get_quadratic_approximation_cost(Z,U, horizon_length,z_bar,u_bar)
state,u,k = controller(Z,U,q,r,horizon_length,Q,R,a)
U = u
Z = np.zeros([14, horizon_length+1])
for i in range(horizon_length):
    Z[:,i+1],var = simulate_system(Z[:,i], U[:,i])
J=compute_cost(Z,U,z_bar,u_bar, horizon_length)
J1=J
print(J)

q,r=get_quadratic_approximation_cost(Z,U, horizon_length,z_bar,u_bar)
state,u,k = controller(Z,U,q,r,horizon_length,Q,R,a)
U = u
Z = np.zeros([14, horizon_length+1])
for i in range(horizon_length):
    Z[:,i+1],var = simulate_system(Z[:,i], U[:,i])
J=compute_cost(Z,U,z_bar,u_bar, horizon_length)
J1=J
print(J)

for j in range(300):
    q,r=get_quadratic_approximation_cost(Z,U, horizon_length,z_bar,u_bar)
    state,u,k = controller(Z,U,q,r,horizon_length,Q,R,a)
    U = u
    Z = np.zeros([14, horizon_length+1])
    for i in range(horizon_length):
        Z[:,i+1],var = simulate_system(Z[:,i], U[:,i])
    J=compute_cost(Z,U,z_bar,u_bar, horizon_length)
    print(J)
    if (J<=J1):
        if (abs(J1-J)<1e-3):
            print("iteration converged")
            break
        J1=J
    else:
        a=a/2
        J1=J
        print(a)

1764.0940700551573
4368.444221190466
3016.3487242705487
10941.381894470414
0.5
568.9304186291665
131.0712586501402
220.08256641730205
0.25
244.69857251536789
0.125
220.22380135918695
266.8308866436493
0.0625
284.0212085196207
0.03125
267.773492788985
278.60551719244836
0.015625
280.57219031950063
0.0078125
270.4187906811664
269.1470017977944
267.8582210214736
256.17405706145536
255.14145076485676
260.583483116799
0.00390625
261.3624641871963
0.001953125
259.72128076337094
259.85643485569904
0.0009765625
259.9263921952759
0.00048828125
259.87635809439837
259.8696809061955
259.66754197437916
259.5599291275139
259.3371595081322
259.1546820786226
260.98470707253847
0.000244140625
260.98283231958544
261.0395155431853
0.0001220703125
261.050684228559
6.103515625e-05
261.05371294193156
3.0517578125e-05
261.04843031828074
261.05280145709946
1.52587890625e-05
261.0517195434448
261.0473686459979
261.0445452199423
261.04289854729853
261.058823144422
7.62939453125e-06
261.0584115994157
iteration c

In [84]:
print(Z[:,-1])
print(z_bar[:,-1])
target = torch.from_numpy(z_bar[:7,-1])
target = target.view(-1,7)
target = target.cuda()
mean = torch.from_numpy(Z[:7,-1])
mean = mean.view(-1,7)
mean = mean.cuda()
mseloss = nn.MSELoss()
output = mseloss(mean, target)
print(output)

[  -1.51067228   -0.95290516   -0.98236871   -0.92318048    0.8210994
    0.29714266   -1.63592916   23.9194169   -15.69344282   -8.43100029
  -22.33611989  -30.81946517  -23.94942093 -291.59596252]
[-1.83364305 -1.49008046 -0.33423962 -1.10883636  0.78155633  0.50425809
  1.14076963  0.          0.          0.          0.          0.
  0.          0.        ]
tensor(1.2288, device='cuda:0', dtype=torch.float64)


In [85]:
print(u)
dataframe = pd.DataFrame(u)
dataframe.to_csv("u.csv")

[[-2.17296983e-01  4.02726727e-02 -7.08119037e-02 -7.19748040e-02
  -5.77669493e-02 -3.33884855e-02 -8.09337247e-02 -8.60744397e-02
  -1.66931416e-02 -3.27173970e-02 -1.04147333e-01  9.81321878e-03
  -7.67249304e-02 -2.70699550e-01 -9.92097876e-02 -1.01524569e-01
   3.41869537e-02  1.29421614e-01 -2.17952808e-01 -7.03779222e-02
  -7.61003955e-02  8.85561968e-02 -2.05930568e-01 -1.06543660e-01
  -8.61523103e-02 -9.46866606e-02  4.02934929e-02 -9.34032222e-02
   2.36215156e-01  6.84697137e-02  3.63036110e-02 -6.46248846e-02
   9.24341183e-02 -8.23033204e-01 -2.30154720e+00  5.68644437e+00
   1.58934743e+00  3.06261865e+01  1.63977679e+01  7.06668199e+01]
 [-2.93129200e-01 -2.92823410e-01 -2.16629683e-01 -2.25572502e-01
  -2.22326329e-01 -4.56418088e-01 -2.33908710e-01 -2.20708252e-01
  -2.98240580e-01 -2.55848540e-01 -2.44725268e-01 -2.67551721e-01
  -2.27170786e-01 -2.40707666e-01 -2.35030796e-01 -2.40236345e-01
  -1.66642738e-01 -4.48256466e-01 -1.96291302e-01 -2.51326100e-01
  -2.3649

---------------------------

In [ ]:
q,r=get_quadratic_approximation_cost(Z,U, horizon_length,z_bar,u_bar)
state,u,k = controller(Z,U,q,r,horizon_length,Q,R,a)
U = u
Z = np.zeros([14, horizon_length+1])
for i in range(horizon_length):
    Z[:,i+1],var = simulate_system(Z[:,i], U[:,i])
J=compute_cost(Z,U,z_bar,u_bar, horizon_length)
J1=J
print(J)

for j in range(100):
    q,r=get_quadratic_approximation_cost(Z,U, horizon_length,z_bar,u_bar)
    state,u,k = controller(Z,U,q,r,horizon_length,Q,R,a)
    U = u
    Z = np.zeros([14, horizon_length+1])
    
    for i in range(horizon_length):
        Z[:,i+1],var = simulate_system(Z[:,i], U[:,i])
    J=compute_cost(Z,U,z_bar,u_bar, horizon_length)
    
    print(J)
    if (J<=J1):
        if (abs(J1-J)<1e-2):
            print("iteration converged")
            break
        J1=J
    else:
        a=a/2
        J1=J
        print(a)

nan


KeyboardInterrupt: ignored

In [ ]:
def get_linearization(current_state, control_signal):
    eps = 1e-5
    A = np.zeros([len(current_state), len(current_state)])
    for ii in range(len(current_state)):
        x = current_state.copy()
        x[ii] += eps
        x_inc,ensemble_varr = simulate_system(x, control_signal) 
        x = current_state.copy()
        x[ii] -= eps
        x_dec,ensemble_varr = simulate_system(x, control_signal)
        A[:,ii] = (x_inc - x_dec) / (2 * eps)
 
    B = np.zeros([len(current_state), len(control_signal)])
    for ii in range(len(control_signal)):
        u = control_signal.copy()
        u[ii] += eps
        x_inc,ensemble_varr = simulate_system(current_state, u)
        u = control_signal.copy()
        u[ii] -= eps
        x_dec,ensemble_varr = simulate_system(current_state, u)
        B[:,ii] = (x_inc - x_dec) / (2 * eps)
    return A, B

def solve_ricatti_equations(Z,U,Q,R,N,q,r):
    
    K_gains = []
    k_feedforward = []
    S = Q[-1].copy()
    s = q[-1].copy()
       
    for i in reversed(range(horizon_length)):
        A, B = get_linearization(Z[:,i], U[:,i])
        #print(ensemble_var)
        W = np.linalg.inv(Sigma)-sigma*C.T.dot(S).dot(C)
        #print("W",W)
        iW = np.linalg.inv(W)
        #print("iW",iW)
        term_covar = C.dot(iW).dot(C.T)
        H = R + B.T.dot(S).dot(B) + sigma * B.T.dot(S.T).dot(term_covar).dot(S).dot(B)
        g = r[i] + B.T.dot(s) + sigma * B.T.dot(S.T).dot(term_covar).dot(s)
        G = B.T.dot(S).dot(A) + sigma * B.T.dot(S.T).dot(term_covar.T).dot(S).dot(A)
        k = -np.linalg.solve(H,g)
        K = -np.linalg.solve(H,G)
        s = q[i] + A.T.dot(s) + G.T.dot(k) + K.T.dot(g) + K.T.dot(H).dot(k) + sigma * A.T.dot(S.T).dot(term_covar).dot(s)
        #print("term_covar",term_covar)
        #print("A.T.dot(S).dot(A)",A.T.dot(S).dot(A))
        #print("K.T.dot(H).dot(K)",K.T.dot(H).dot(K))
        #print("G.T.dot(K)",G.T.dot(K))
        #print("K.T.dot(G)",K.T.dot(G))
        #print("A",A)
        #print("S",S)
        #print("K",K)
        #print("G",G)
        #print("Q[i]",Q[i])
        #print("term_covar",term_covar)
        #print("sigma * A.T.dot(S.T).dot(term_covar).dot(S).dot(A)", A.T.dot(S.T).dot(term_covar).dot(S).dot(A))
        S = Q[i] + A.T.dot(S).dot(A) + K.T.dot(H).dot(K) + G.T.dot(K) + K.T.dot(G) + sigma * A.T.dot(S.T).dot(term_covar).dot(S).dot(A)
        #print("",S)
        K_gains.append(K)
        k_feedforward.append(k)
        
       
    K_gains = K_gains[::-1]
    k_feedforward = k_feedforward[::-1]
    return K_gains, k_feedforward

def controller(Z,U,q,r,horizon_length,Q,R,alpha):
    state2=np.empty([14, horizon_length+1])
    z0 = np.zeros([14,])
    state2[:,0] = z0
    u2 = np.zeros([7, horizon_length])
    K,k = solve_ricatti_equations(Z,U,Q,R,horizon_length,q,r)
    for i in range(horizon_length):
        u2[:,i] = U[:,i] + K[i] @ (state2[:,i] - Z[:,i])+ alpha*k[i]
        state2[:,i+1],ensemble_varr = simulate_system(state2[:,i], u2[:,i])
   
    state=state2.copy()
    u=u2.copy()
    return state,u,k

def compute_cost(Z,U,z_bar,u_bar, horizon_length):
    J=((Z[:,horizon_length]-z_bar[:,horizon_length]).T)@ Q[horizon_length] @(Z[:,horizon_length]-z_bar[:,horizon_length])
    for i in range(horizon_length):
        J=J+((Z[:,i]-z_bar[:,i]).T)@ Q[i] @(Z[:,i]-z_bar[:,i])+((U[:,i]-u_bar[:,i]).T @ R @ (U[:,i]-u_bar[:,i]))
    
    return J

def get_quadratic_approximation_cost(Z,U, horizon_length,z_bar,u_bar):
    q=[]
    r=[]
    J=compute_cost(Z,U,z_bar,u_bar, horizon_length)
    #J=J+0.5*((z[:,horizon_length]-Z[:,horizon_length]).T)@ Q[horizon_length] @(z[:,horizon_length]-Z[:,horizon_length])+2*(z[:,horizon_length].T-z_bar[:,horizon_length].T)@Q[horizon_length]@(z[:,horizon_length]-Z[:,horizon_length])
    for i in range(horizon_length):
        #J=J+0.5*((u[:,i]-U[:,i]).T)@ R @(u[:,i]-U[:,i])+0.5*((z[:,i]-Z[:,i]).T)@ Q[i] @(z[:,i]-Z[:,i])+2*((z[:,i]-z_bar[:,i]).T)@Q[i]@(z[:,i]-Z[:,i])+2*((u[:,i]-u_bar[:,i]).T)@R@(u[:,i]-U[:,i])    
        q1=2*(Z[:,i]-z_bar[:,i]).T@Q[i]
        q.append(q1)
        r1=2*(U[:,i]-u_bar[:,i]).T@R
        r.append(r1)
    return q,r

In [ ]:
a=1
horizon_length = 40
Q2=0.001*np.eye(14)
Q1=20*np.eye(14)
for i in range(7,14):
    Q1[i,i]=0.1
Q=[]
for i in range(horizon_length+1):
    if (i==horizon_length):
        Q.append(Q1)
    else:
        Q.append(Q2)

R=0.001*np.eye(7)
sigma=0
C=np.eye(14)
Sigma=1*np.eye(14)


u_bar = np.zeros([7, horizon_length])
U = np.zeros([7, horizon_length])
Z = np.zeros([14,horizon_length+1])
u= np.zeros([7, horizon_length])
    
      
z_bar=np.zeros([14,horizon_length+1])
z_bar[0:7,horizon_length]=np.array([-1.8336430453226058, -1.4900804580013904, -0.33423961967431476, -1.108836361791467, 0.7815563306180698, 0.5042580934421823, 1.1407696303315698])
z_bar[0:7,horizon_length-1]=np.array([-1.8336430453226058, -1.4900804580013904, -0.33423961967431476, -1.108836361791467, 0.7815563306180698, 0.5042580934421823, 1.1407696303315698])